# RBF Regression on Boston hosting Dataset
### Charis Filis
### Academic-id : 9449 

Add needed dependencies

In [182]:
import tensorflow as tf
# Import  tensorflow.keras basic NNet modules for the RBF output layer
from tensorflow.keras import layers,models
from tensorflow.keras import initializers
# Import tensorflow.keras backend for any tensor operations
from tensorflow.keras import backend as K

from tensorflow.keras.datasets import boston_housing
# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# import addons for RSqure metric
import tensorflow_addons as tfa
from tensorflow.keras.metrics import RootMeanSquaredError
# Import pairwise metric 
from sklearn.metrics import pairwise
# Import KMeans algorithm in case the custom one does not work properly
from sklearn.cluster import KMeans
# Import numpy for array and numeric operations
import numpy as np

In [183]:
(x_train, y_train),(x_test, y_test) = boston_housing.load_data(test_split=0.25)


#Convert to float32
x_train, x_test = np.array(x_train, dtype=np.float32), np.array(x_test, dtype=np.float32)
print(x_train[1], y_train[1])
# get per-feature statistics (mean,standard deviation ) from the training set in order to make normalization
train_mean = np.mean(x_train, axis=0)
train_std = np.std(x_train,axis=0)
x_train = (x_train-train_mean)/train_std
print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))
print("Training data shape:",x_train.shape)
print("Training y train shape", y_train.shape)
print("Testing data shape:",x_test.shape)


[2.1770e-02 8.2500e+01 2.0300e+00 0.0000e+00 4.1500e-01 7.6100e+00
 1.5700e+01 6.2700e+00 2.0000e+00 3.4800e+02 1.4700e+01 3.9538e+02
 3.1100e+00] 42.3
Number of original training examples: 379
Number of original test examples: 127
Training data shape: (379, 13)
Training y train shape (379,)
Testing data shape: (127, 13)


R squared metric custom made 

In [184]:
def R2(test_val, pred_val):
    SS_res = K.sum(K.square(test_val-pred_val))
    SS_tot = K.sum(K.square(test_val-K.mean(test_val)))
    return (1 - SS_res/(SS_tot+K.epsilon()))

Model 1 compute centers and sigma and Number of Kernels = 0.1 Ntrain

In [185]:
from scipy.spatial.distance import pdist
# Variance of each kernel
from cmath import sqrt
# num of kernels in hidden layer = number of inputs = number of nodes of  RBF layer
n_centers = int(0.1*x_train.shape[0])

def computeSigma(n_centers):
        km = KMeans(n_clusters=n_centers, init='random', verbose=0).fit(x_train)
        centers = km.cluster_centers_
        dists = pdist(centers, metric='euclidean')
        d_max = np.amax(dists)
        sigma = d_max / tf.math.sqrt(2*float(centers.shape[0])) 
        return sigma
   
sigma = computeSigma(n_centers)
print(sigma)

tf.Tensor(1.340571, shape=(), dtype=float32)


In [1]:
# custom Kmeans algorithm that returns cluster centers and standart deviation of each cluster
"""
1D-Input Clustering
A := numpy array MX1 - Data
k := integer - Number of Clusters
"""
def kmeans_custom(A,k):
    
    # get Number of Features
    numFeatures = A.shape[1]
    # random initialization of centroids
    clusters_c = np.random.choice(np.squeeze(A),size=k)
    # kmeans is a greedy algorithm so we need a converge flag to stop him from running infinately
    converge_flag = False
    # initialize standart deviation buffer
    stds_c = np.zeros(k)
    
    while not converge_flag:
        """
            Compute distances of each cluster center to each point 
            of the dataset
        """
        distances = np.squeeze(np.abs(A[:,np.newaxis]- clusters_c[np.newaxis, :]))
        # Find the cluster that is closest to each point
        closer_cluster = np.argmin(distances,axis=1)
        
        for iter in range(k):
            clusterSet = A[closer_cluster == iter]
            if len(clusterSet) >  0:
                clusters_c
        

SyntaxError: incomplete input (574062755.py, line 19)

In [186]:
from keras.initializers import Initializer
from sklearn.cluster import KMeans


class InitCentersKMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: matrix, dataset
    """

    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]

        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0)
        km.fit(self.X)
        return km.cluster_centers_
   
        

Create RBF layer or RBF kernel would be described

In [190]:
from tensorflow.keras.layers import Layer


class RBFLayer(Layer):
    def __init__(self,units,initializer,sigmaInit,**kwards):
        super(RBFLayer,self).__init__(**kwards)
        self.units = units
        self.sigmaInit = sigmaInit
        self.initializer= initializer
    def build(self, input_shape):
        # I initialize the weights of rbf layer with kmeans 
        self.W = self.add_weight(name='W',
                                 shape=(int(input_shape[1],), self.units),
                                 initializer=self.initializer,
                                 trainable=True)
        self.b  = self.add_weight(shape=(self.units,),
                            initializer=self.initializer,
                            trainable=True)

        super(RBFLayer,self).build(input_shape)
    
    # radial basis function
    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.W
        # take L2 norm for the exponent of the radial basis function
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp((-1 * l2)/(2*(self.sigmaInit**2)))     
        return tf.matmul(res.transpose(), self.W) + self.b

    def compute_output_shape(self, input_shape):
        return (input_shape, self.units)
                                 
        
    
    

### Create 1st model builder function

In [191]:
def build_model(models):
    model_out = models.Sequential()
    # RBF layer
    model_out.add(RBFLayer(units = (int)(0.1*x_train.shape[0]),initializer=InitCentersKMeans(x_train),sigmaInit=sigma))
    # Output Layer
    rbf_out_shape = RBFLayer(units = (int)(0.1*x_train.shape[0]),initializer=InitCentersKMeans(x_train),sigmaInit=sigma).compute_output_shape(input_shape=((int)(x_train.shape[1],)))
    model_out.add(layers.Dense(128,activation='relu',input_shape=rbf_out_shape,
                               kernel_initializer=initializers.RandomNormal(mean = 0,stddev=0.4)))
    
    # Make the linear tranformation layer on the output layer
    model_out.add(layers.Dense(1,activation='relu',
                  kernel_initializer= initializers.RandomNormal(mean = 0,stddev=0.4)))
    model_out.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0),
                      loss= [RootMeanSquaredError, tfa.metrics.RSquare])
    

Train output layer

In [195]:
model_out = models.Sequential()
model_out.add(layers.Dense(0.1*x_train.shape[0], input_shape=(x_train.shape[0],)))
model_out.add(RBFLayer(units = (int)(0.1*x_train.shape[0]),initializer=InitCentersKMeans(shape=(x_train.shape[0],)),sigmaInit=sigma))
# Output Layer
rbf_out_shape = RBFLayer(units = (int)(0.1*x_train.shape[0]),initializer=InitCentersKMeans(shape=(x_train.shape[0],)),sigmaInit=sigma).compute_output_shape(input_shape=((int)(x_train.shape[1],)))
model_out.add(layers.Dense(128,activation='relu',input_shape=rbf_out_shape,
                            kernel_initializer=initializers.RandomNormal(mean = 0,stddev=0.4)))

# Make the linear tranformation layer on the output layer
model_out.add(layers.Dense(1,activation='relu',
                kernel_initializer= initializers.RandomNormal(mean = 0,stddev=0.4)))
model_out.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001),
                    loss = tf.keras.losses.MeanSquaredError(),
                    metrics = [tf.keras.metrics.RootMeanSquaredError(), R2])
history_1 = model_out.fit(x_train,y_train,epochs=100,batch_size=128,validation_split=0.2,verbose=1)


TypeError: __init__() got an unexpected keyword argument 'shape'

### Reconsideration
At this point i realize that keras models are not working well with custom made layers such as the one RBF I made. After research and communication with a professor,
I came to an understanding that RBF layer is just a kernel that could be applied to data and the we can instantly proceed adn push that data to the output layer.
So because I have no time for debugging im using sklearn to create a custom rbf kernel from pairwise lib and I apply this to a specific number of data inputs in order to have the correct. Last but not least I want to point out that there are implementation of creating rbf Layers that work with keras and I found some but they are not quite effective.

In [123]:
from sklearn.metrics.pairwise import rbf_kernel

# apply kmeans to find initial centers
km = KMeans(n_clusters=n_centers, init='random', verbose=0).fit(x_train)
centers = km.cluster_centers_
# typecast 1/2sigma to float vector in order to be applied to kernel
x_rbf_out = rbf_kernel(x_train,Y =centers, gamma = np.float32( 1/(2*sigma)) )


### Create output layer model builder function

In [179]:
def output_layer_builder():
    model_out = models.Sequential()
    # RBF layer kernel applied to data as transformation
    # Output Layer
    model_out.add(layers.Dense(128, activation='relu', input_shape=(x_rbf_out.shape[1],),
                         kernel_initializer=initializers.RandomNormal(mean = 0,stddev=0.4)))
    
    # Make the linear tranformation layer on the output layer
    model_out.add(layers.Dense(1,activation='relu',
                  kernel_initializer= initializers.RandomNormal(mean = 0,stddev=0.4)))
    model_out.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001),
                    loss = tf.keras.losses.MeanSquaredError(),
                    metrics = [tf.keras.metrics.RootMeanSquaredError(), R2])

Train Model 1

In [180]:

model_1 = output_layer_builder()
history_1 = model_1.fit(x_rbf_out,y_train,epochs=100,batch_size=48,validation_split=0.2,verbose=1)

# Plot RSQ
plt.figure(1)
plt.plot(history.history['RSquare'])





# Plot RMSE
plt.figure(1)
plt.plot(history.history['root_mean_squared_error'],label='train')
plt.plot(history.history['val_root_mean_squared_error'],label='test')
plt.legend()
plt.title('RMSE on training and validation sets')
plt.show()

AttributeError: 'NoneType' object has no attribute 'fit'